In [1]:
#импорт библиотек
import pandas as pd
import numpy as np

In [2]:
address_clust = pd.read_csv('address_clust.csv')
address_stats = pd.read_csv('address_stats.csv')
address_stats = address_stats.drop_duplicates().reset_index(drop=True)
new_clusters = address_stats.merge(address_clust, on = 'address_id', how = 'outer')
new_clusters['cluster_id'] = new_clusters['cluster_id'].fillna(value='0').astype(int)

In [3]:
tranzactions = new_clusters.groupby('transaction_id').agg({'id': 'count'}).reset_index()
tranzactions['transaction_id']=tranzactions['transaction_id'].astype(str)
tranzactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13753 entries, 0 to 13752
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   transaction_id  13753 non-null  object
 1   id              13753 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 215.0+ KB


***Тут сделали группировку по transaction_id, второй столбец для массовки и проверки всё ли посчитали.
transaction_id у нас ключевой столбец, по нему считается всё.***

In [4]:
def clusters(row):
    """Функция для подсчета сколько биткоинов отправлено из одного кластера в другой.

    Функция принимает на вход строку с transaction_id и далее считает сумму
    для каждой транзакции по кластерам. Далее считаются доли отправленных и
    полученных биткоинов. В конце количество отправленных умножается на долю полученных
    """

    transaction_id = row['transaction_id']
    tranz = new_clusters.query('transaction_id=='+transaction_id)
    s1= tranz.query('cluster_id==1')['sent'].sum() #сумма отправленных в кластере 1
    s2=tranz.query('cluster_id==2')['sent'].sum() #сумма отправленных в кластере 2
    s0=tranz.query('cluster_id==0')['sent'].sum() #сумма отправленных в кластере 0
    r1= tranz.query('cluster_id==1')['received'].sum() #сумма полученных в кластере 1
    r2= tranz.query('cluster_id==2')['received'].sum() #сумма полученных в кластере 2
    r0= tranz.query('cluster_id==0')['received'].sum() #сумма полученных в кластере 0
    
    if from_clust==1 and s1==0: #если нет отправленных, то не учитывыется при подсчете
        return 0
    if from_clust==2 and s2==0: #если нет отправленных, то не учитывыется при подсчете
        return 0
    if from_clust==0 and s0==0: #если нет отправленных, то не учитывыется при подсчете
        return 0
    if to_clust==1 and r1==0: #если нет полученных, то не учитывыется при подсчете
        return 0
    if to_clust==2 and r2==0: #если нет полученных, то не учитывыется при подсчете
        return 0
    if to_clust==0 and r0==0: #если нет полученных, то не учитывыется при подсчете
        return 0    
    
    if from_clust==1: #количество отправленных в кластере 1 умноженное на долю отправленных этим кластером 
        result=s1*(s1/(s1+s2+s0))
    if from_clust==2: #количество отправленных в кластере 2 умноженное на долю отправленных этим кластером 
        result=s2*(s2/(s1+s2+s0))
    if from_clust==0: #количество отправленных в кластере 0 умноженное на долю отправленных этим кластером 
        result=s0*(s0/(s1+s2+s0))
    if to_clust==1: #доля полученных в кластере 1
        k_rec=r1/(r1+r2+r0)
    if to_clust==2: #доля полученных в кластере 2
        k_rec=r2/(r1+r2+r0)
    if to_clust==0: #доля полученных в кластере 0
        k_rec=r0/(r1+r2+r0)
        
    return result*k_rec #умножение

Итак, мы задаем 2 переменные:
- from_clust из какого кластера отправлено
- to_clust в какой кластер отправлено

Далее из полученной таблицы с transaction_id берем кажду строку и передаем ее в функцию.

Объявляем переменные для суммы полученных и отправленных в каждом кластере.

Пишем условия, где значение будет 0(чтобы точно верную сумму получить).

Далее получаем количество отправленных в кластерах и доли полученных в кластерах и перемножаем.

In [5]:
from_clust=1
to_clust=2
tranzactions['column_one'] = tranzactions.apply(clusters, axis=1)
print('Сатоши из кластера 1 в кластер 2 отправлено: ',tranzactions.column_one.sum())

Сатоши из кластера 1 в кластер 2 отправлено:  6473247884.412515


In [6]:
from_clust=2
to_clust=1
tranzactions['column_two'] = tranzactions.apply(clusters, axis=1)
print('Сатоши из кластера 2 в кластер 1 отправлено: ',tranzactions.column_two.sum())

Сатоши из кластера 2 в кластер 1 отправлено:  6806727577.694637


In [7]:
from_clust=1
to_clust=0
tranzactions['column_three'] = tranzactions.apply(clusters, axis=1)
print('Сатоши из кластера 1 в кластер 0 отправлено: ',tranzactions.column_three.sum())

Сатоши из кластера 1 в кластер 0 отправлено:  29167612941.802597


In [8]:
from_clust=0
to_clust=1
tranzactions['column_four'] = tranzactions.apply(clusters, axis=1)
print('Сатоши из кластера 0 в кластер 1 отправлено: ',tranzactions.column_four.sum())

Сатоши из кластера 0 в кластер 1 отправлено:  28916274183.57309


In [9]:
from_clust=2
to_clust=0
tranzactions['column_five'] = tranzactions.apply(clusters, axis=1)
print('Сатоши из кластера 2 в кластер 0 отправлено: ',tranzactions.column_five.sum())

Сатоши из кластера 2 в кластер 0 отправлено:  243721256.44623548


In [10]:
from_clust=0
to_clust=2
tranzactions['column_six'] = tranzactions.apply(clusters, axis=1)
print('Сатоши из кластера 0 в кластер 2 отправлено: ',tranzactions.column_six.sum())

Сатоши из кластера 0 в кластер 2 отправлено:  610432873.9895253


In [11]:
#выведем fee в отдельный столбец
fee = new_clusters.groupby(['transaction_id','cluster_id']).agg({'received':'sum', 'sent':'sum'}).reset_index()
fee['fee'] = fee['sent']-fee['received']
display(fee.head())

,transaction_id,cluster_id,received,sent,fee
0,19162323,0,1821052.0,2871052.0,1050000.0
1,19162323,1,1000000.0,0.0,-1000000.0
2,19166856,0,990000.0,0.0,-990000.0
3,19166856,1,0.0,1000000.0,1000000.0
4,19174421,0,4590000.0,4800000.0,210000.0


In [12]:
print('Кластер 1 потратил на fee:', (fee.query("cluster_id==1 & fee >= 0")['fee']).sum())
print('Кластер 2 потратил на fee:', (fee.query("cluster_id==2 & fee >= 0")['fee']).sum())

Кластер 1 потратил на fee: 35663479860.0
Кластер 2 потратил на fee: 7071802861.0
